Para a raspagem do Mercado Livre, foi utilizado um código para a primeira página e copiado para gerar da segunda página em diante. 
Na verdade a estrutura do codigo são iguais a mudança ocorre na construção da url.

Assim, este notebook foi divido colocando um bloco de código para cada termo pesquisado (brinquedo, escritório..) e dentro deste bloco um bloco para a primeira página e outro para as páginas seguintes (página 2 em diante).

As funções de extrair primeira página e páginas seguintes não foram terminadas, o que deixaria o código mais enxuto e legível.

A função extrai_valor_parcelas esta em aplicação, ela é usada após a raspagem para tratar a coluna Parcelas que na extração possui a quantidade de parcelas e o valor juntos, esta função cria mais 2 colunas com estes respectivos valores.


## URL de pesquisa
A estrututa do url da primeira página é simples: composto pela url da homepage mais o item pesquisado conforme abaixo:

homepage  = "https://lista.mercadolivre.com.br/"
item pesquisa = 'brinquedo'
url da pesquisa = "{homepage}{item}#D[A:{item}]"

Exemplo: https://lista.mercadolivre.com.br/notebook#D[A:notebook]

Nas paginas seguintes a estrutura muda, é inserido na estrutura a categoria do item, sendo necessário pesquisar previamente, e o número da páginas dentro do url segue um adrão diferente para cada pesquisa. Exemplo, na segunda página, aparece dentro da url  o número 49 e as para as páginas seguintes soma-se 48. Mas, para outros termos a segunda página pode ser 50, e as seguintes soma-se 51. Assim, é necessário uma pesquisa anterior para realizar a raspagem. Mas, a estrutura das urls seguem estes padrões.
Segue exemplos abaixo:

página 2 da pesquisa de brinquedo: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_49_NoIndex_True  
página 3: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_97_NoIndex_True


Os códigos apresentam breves comentarios para melhor entendimento.


## Patrocinados

Os produtos patrocinados aparecem em cada página, e estão em uma classe do código html diferente dos produtos normais, assim, dentro do bloco de código da Primeira página e de Páginas seguintes a primeira raspagem são nestes produtos e depois nos normais. 


# COMO PESQUISAR NESTA RASPAGEM

- Rodar a célula de importação de bibliotecas e da função extrai_valor_parcelas

- Na célula da Raspagem:
    - alterar a variável 'item' para o termo que deseja ser pesquisado
    - (se necessário) alterar a variavel 'n' para o numero que aparece na url da pagina 2 (ver URL de pesquisa mais acima).
    - (se necessário) alterar a variável incremento, verificar o incremento da url da pagina 2 para pagina 3.
    * Obs. A maioria dos itens pesquisados o n é 49 e incremento 48, mas pode haver pesquisa que n é 51 e incremento 50.
    - alterar a variável 'search_url2' para a url da página 2, dentro desta url trocar o numero 49 (ou 51, algumas vezes) para {n} (letra n entre chaves).
    - alterar a variavel 'caminho' para o endereço da pasta e nome do arquivo xlsx que ficará salvo a planilha.

Pronto! Rodar todas as células, depois de alguns minutos o arquivo xlsx aparecerá na pasta indicada no 'caminho'.

# 1.0 Biblioteca

In [164]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import openpyxl
import numpy as np
from datetime import datetime



# 2.0 Função 


In [165]:
# função que separa o conteudo da coluna Parcelas (numero de parcelas e valor)
# e realiza a conta do valor total das parcelas

def extrai_valor_parcelas(df):

    # separa o valor de parcela utilizando expressão regular. extrai o valor com virgula que constam após o R$
    df['Valor Parcelas'] = df['Parcelas'].str.extract(r'R\$(\d+,\d+)')
    df['Valor Parcelas'] = df['Valor Parcelas'].fillna(0)
    df['Valor Parcelas'] = df['Valor Parcelas'].apply(lambda x: float(str(x).replace(',', '.')))

    # as parcelas aparecem ao lado da letra x como: '12x'
    # expressão que separa o numeros seguidos por 'x' 
    df['N Parcelas'] = df['Parcelas'].str.extract(r'(\d+)x')
    df['N Parcelas'] = df['N Parcelas'].fillna(0)
    df['N Parcelas'] = df['N Parcelas'].apply(lambda x: int(x))

    df[' Total parcelas'] = df['Valor Parcelas'] * df['N Parcelas']

    return df

# 2.0 Primeira Pagina 

In [166]:
### ALTERAR para o item que deseja buscar
item = 'pu 40 selante sc'


# numero do url pagina 2
n = 51

# incremento por pagina
incremento = 50
## ALTERAR o url da pagina 2, e no lugar do numero 49 colocar {n}
search_url2 = f'https://lista.mercadolivre.com.br/pu-40-selante-sc_Desde_{n}_NoIndex_True'
caminho = f'C:/Users/Leticia/Documents/raspagem atualizada/datafreme/pu 40 selante sc.xlsx'

In [167]:
#montando a URL de pesquisa
base_url = "https://lista.mercadolivre.com.br/"


if ' ' in item:
    search_url = f"{base_url}{item.replace(' ', '-')}#D[A:{item.replace(' ', '%20')}]"

else:
    search_url = f"{base_url}{item}#D[A:{item}]"

### 2.1 Verificando a existencia da pagina

In [168]:
# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

In [169]:
# Parsing do HTML 
soup = BeautifulSoup(response.text, 'html.parser')

# Classe html dos produtos normais
# Encontra os elementos que contêm as informações do produto
# todos os produtos tem essa classe, então usa-se o find all para trazer todos
products = soup.find_all(class_='ui-search-result__content-wrapper')


# Lista para armazenar os dados dos produtos patrocinados
products_data_patr = []

# Lista para armazenar os dados dos produtos normais
products_data = []

### 2.2 Buscando todos os produtos que são patrocidados 

In [170]:
# Classe html dos produtos patrocinados
# Encontra os elementos que contêm as informações do produto
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')

df_patr = pd.DataFrame()

if products_patr !=  None:
    
    products_patr = products_patr.find_all('li', class_='ui-search-layout__item')
    




    for i in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text

        try:
        # avaliação
            product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number').text
            
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
        except:
            product_qt_eval = ''
        
           

        try: 
            # parcelas
            product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.text
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
        except:
            product_discount = ''


        #link
        product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href que está o link do produto
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            
            info_element = soup2.find('div', class_='ui-pdp-seller__header__title')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass

        ##############################################################################################
        # posição de venda na categoria, se possui
    
        try:
            position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
            if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
                
                position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
                position = position[1].text
            else:
                position = ''
        except:
             position = ''                   
 

            
        # Frete
        try:
            product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
            
        except:
            product_fret = ''



        try:
        # Novo/usado e qtde vendida
            
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass


            # Quant. de fotos e data foto
        try:
            
            # grade que estao as fotos
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')
            
            
            # quantidade de fotos
            images = len(images)
        except:
            pass


        # data da foto
            
        # date_image = soup2.find('figure', class_ = 'ui-pdp-gallery__figure')
        
        # #date_image = date_image.find('img')
        
        # date_image = re.findall(r'https://\S+', str(date_image))[0]
       
        url_list = []
        date_images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')    
        for img in date_images:
            date_images_url =  re.findall(r'https://\S+', str(img))
            url_list.append(date_images_url[0])
        

        
        

    #CATEGORIAS
    # #########################################################################################################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
        
        
        patrocinado = 'sim'
    ###############################################

        
        # Adiciona os dados do produto à lista
        products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                            'Categorias': categories_list,'Quant. fotos': images, 'HTML Data':url_list })

### 2.2.Produtos NAO patrocinados 

In [171]:
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    # se patrocinado caso, pagina seja com produtos na vertical
    product_patr = products[i].find('label', class_='ui-search-styled-label ui-search-item__pub-label')
        
    if product_patr == None:
        patrocinado = 'nao'

    else:
        patrocinado = 'sim'
    
       
    try:
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        product_eval = ''
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span',class_='ui-search-reviews__amount').text.strip()
    except:
        product_qt_eval=''
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        product_discount = ''


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        info_element = soup2.find('div', class_='ui-pdp-seller__header__title')
        # Extrai o vendedor 
        seller = info_element.text
    except:
        pass
    
    
    # posição de venda na categoria, se possui
    
    try:
        position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
        if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
            
            position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text
        else:
            position = ''
    except:
        position = ' '

    # Frete
    try:
        product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
        
    except:
        product_fret = ''


    try:
    # Novo/usado e qtde vendida
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')
        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]
        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')
        images = len(images)
    except:
        pass

    #data da foto           
    url_list = []
    date_images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')    
    for img in date_images:
        date_images_url =  re.findall(r'https://\S+', str(img))
        url_list.append(date_images_url[0])
    
    
    # categorias
    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])


    #criando a lista de produtos 

    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': url_list})

    # Cria um DataFrame com os dados dos produtos
    df = pd.DataFrame(products_data)

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\845299142.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  product_juros = product_juros.find(text='juros')


# 3.0 Coletando dados da 1 Pagina a 5 Pagina 

In [172]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    

    # Realiza a requisição HTTP
    response = requests.get(search_url2)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    

    if products_patr !=  None:
        
        products_patr = products_patr.find_all('li', class_='ui-search-layout__item')

        for x in range(len(products_patr)):

            
            # Obtém o nome do produto
            product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
            
            # Obtém o preço do produto
            product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

            try:
            # avaliação
                product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
            except:
                pass

            try:
            # qtde avaliações
                product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
            except:
                pass
            
            #frete gratis
            try:
                product_fret = products_patr[x].find('span', class_='ui-pb-highlight').text.striP()
            except:
                product_fret = ''    

            try: 
                # parcelas
                product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

                # Extrai o texto do elemento
                product_parc = product_parc.get_text(strip=True)
            except:
                pass

            # sem juros    
            try:
                product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
                product_juros = product_juros.find(text='juros')

            except:
                pass
                # desconto
            try:
                product_discount = products_patr[x].find('span', class_='ui-search-price__discount').text.strip()
            except:
                product_discount = ''


            #link
            product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

            # Extrai o atributo href do elemento
            product_link = product_link['href']


            # DADOS LINK

            response2 = requests.get(product_link)

            # Verifica se a requisição foi bem-sucedida
            if response2.status_code != 200:
                print(f"Erro ao acessar a URL: {product_link}")

            # Parsing do HTML
            soup2 = BeautifulSoup(response2.text, 'html.parser')


            # Nome vendedor
            try:
                #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
                #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
                #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
                info_element = soup2.find('div', class_='ui-pdp-seller__header__title')

                # Extrai o vendedor 
                seller = info_element.text

            except:
                seller= ''

            # posição de venda na categoria, se possui
            
            try:
                position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
                if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
                    
                    position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
                    position = position[1].text
                else:
                    position = ''
                    
            except:
                position = ''

            # Frete
            try:
                product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
                
            except:
                product_fret = ''


            try:
            # Novo/usado e qtde vendida
                situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')

                # Separa as informações 'Novo' e '+500 vendidos'
                info_parts = situation.text.split('|')

                # Remove os espaços em branco ao redor de cada parte
                info_parts = [part.strip() for part in info_parts]

                product_situation = info_parts[0]
                product_sells = info_parts[1]
            except:
                pass

                # Quant. de fotos
            try:
 
                images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

                images = len(images)
            except:
                pass
          
            url_list = []
            date_images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')    
            for img in date_images:
               date_images_url =  re.findall(r'https://\S+', str(img))
               url_list.append(date_images_url[0])

            # categorias
            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                cat = cat.find('a', class_ = 'andes-breadcrumb__link')
                categories_list.append(cat['title'])
        
    ###############################################

        
            
        
            patrocinado = 'sim'
            
            # Adiciona os dados do produto à lista
            all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                            'Categorias': categories_list,'Quant. fotos': images, 'HTML Data':url_list })

    # Cria um DataFrame com os dados dos produtos

        patr_temp_df = pd.DataFrame(all_products_data_patr)
        all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
        



##########################################################################################################################3






        for product in products:
            # Obtém o nome do produto
            product_name = product.find('h2', class_='ui-search-item__title').text.strip()

            #preço
            try:
                product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()
            except:
                product_price = product.find('span', class_='andes-money-amount__fraction').text.strip()


            # se patrocinado caso, pagina seja com produtos na vertical
            product_patr = products[i].find('label', class_='ui-search-styled-label ui-search-item__pub-label')
            

            
            

            if product_patr == None:
                patrocinado = 'nao'
                
            else:
                patrocinado = 'sim'
                


            try:
        # avaliação
                product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
            except:
                pass

            try:
            # qtde avaliações
                product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
            except:
                pass
            
            #frete gratis
            try:
                product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
            except:
                product_fret = ''    


            try: 
            # parcelas
                product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

                # Extrai o texto do elemento
                product_parc = product_parc.get_text(strip=True)
            except:
                pass

            # sem juros    
            try:
                product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
                product_juros = product_juros.find(text='juros')

            except:
                pass
            # desconto
            try:
                product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
            except:
                product_discount = ''


            

            #link
            product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

            # Extrai o atributo href do elemento
            product_link = product_link['href']


            # DADOS LINK

            response2 = requests.get(product_link)

            # Verifica se a requisição foi bem-sucedida
            if response2.status_code != 200:
                print(f"Erro ao acessar a URL: {search_url}")

            # Parsing do HTML
            soup2 = BeautifulSoup(response2.text, 'html.parser')

            # Vendedor
            try:
                info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

                # Extrai o texto do elemento
                seller = info_element
            except:
                pass

            
            # posição de venda na categoria, se possui
        
            try:
                position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
                if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
                    
                    position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
                    position = position[1].text

                else:
                    position = ''
                    
            except:
                position = ''

            # Frete
            try:
                product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
                
            except:
                product_fret = ''
                

                # Novo/usado e qtde vendida
            try:
            
                #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


                # Separa as informações 'Novo' e '+500 vendidos'
                info_parts = situation.text.split('|')

                # Remove os espaços em branco ao redor de cada parte
                info_parts = [part.strip() for part in info_parts]

                product_situation = info_parts[0]
                product_sells = info_parts[1]
            except:
                pass

                # Quant. de fotos
            try:
                images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

                images = len(images)
            except:
                pass

            # data da foto
                url_list = []
                date_images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')    
                for img in date_images:

                    date_images_url =  re.findall(r'https://\S+', str(img))
                    url_list.append(date_images_url[0])


    #####################################

            # categorias
            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                cat = cat.find('a', class_ = 'andes-breadcrumb__link')
                categories_list.append(cat['title'])
        
    ###############################################

            # Adiciona os dados do produto à lista
            products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                            'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': url_list})



    ####    # Incrementa a posição conforme a página seguinte
        
        n = n + incremento
        # Cria um DataFrame temporário com os dados dos produtos desta iteração
        temp_df = pd.DataFrame(products_data)

        # Concatena o DataFrame temporário ao DataFrame principal
        all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)




    # juntando os dataframes
    df_mercado_brinquedo = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
    df_mercado_brinquedo = extrai_valor_parcelas(df_mercado_brinquedo)
    df_mercado_brinquedo.drop(['Parcelas'], axis = 1, inplace = True)

    df_mercado_brinquedo['Categoria'] = item



    df_mercado_brinquedo['Categoria0'] = 0
    df_mercado_brinquedo['Categoria1'] = 0
    df_mercado_brinquedo['Categoria2'] = 0
    df_mercado_brinquedo['Categoria3'] = 0
    df_mercado_brinquedo['Categoria4'] = 0
    df_mercado_brinquedo['Categoria5'] = 0
    df_mercado_brinquedo['Categoria6'] = 0

    for i in range(len(df_mercado_brinquedo['Categorias'])):

        for x in range(len(df_mercado_brinquedo['Categorias'].iloc[i])):
            df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo['Categorias'].iloc[i][x]

            
            
    for i in range(len(df_mercado_brinquedo['Categorias'])):

            for x in range(1,7):
                if df_mercado_brinquedo[f'Categoria{x}'].iloc[i] == 0:
                    df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo[f'Categoria{x-1}'].iloc[i]




   


C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\3805181249.py:426: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo['Categorias'].iloc[i][x]
C:\Users\Leticia\AppData\Local

In [173]:
### Exportando o dataframe para planlha excel
#df_mercado_brinquedo.to_excel(caminho, index=False)


# 4.0 Limpeza dos dados 

In [174]:
#copiando o dataset
df= df_mercado_brinquedo.copy()


In [175]:
df.shape

(52, 26)

In [176]:
#verificando o nome das colunas
df.columns

Index(['Nome do Produto', 'Preço', 'Patrocinado', 'Avaliação',
       'Qtde avaliações:', 'Desconto', 'Frete', 'Link', 'Vendedor', 'Posição',
       'Situação', 'Quant. vendida', 'Categorias', 'Quant. fotos', 'HTML Data',
       'Valor Parcelas', 'N Parcelas', ' Total parcelas', 'Categoria',
       'Categoria0', 'Categoria1', 'Categoria2', 'Categoria3', 'Categoria4',
       'Categoria5', 'Categoria6'],
      dtype='object')

In [177]:
df2= df.copy()

In [178]:
#verificandoo tamanho do datafreme 
df2.head()

,Nome do Produto,Preço,Patrocinado,Avaliação,Qtde avaliações:,Desconto,Frete,Link,Vendedor,Posição,...,N Parcelas,Total parcelas,Categoria,Categoria0,Categoria1,Categoria2,Categoria3,Categoria4,Categoria5,Categoria6
0,Silicone Pu40 Adesivo Selante Poliuretano Cola...,"R$123,40",nao,,,5% OFF,Frete grátis,https://produto.mercadolivre.com.br/MLB-371236...,Loja oficialNinja Parts SC,,...,12,143.88,pu 40 selante sc,0,0,0,0,0,0,0
1,Silicone Cola Pu40 Selante Adesivo Poliuretano...,"R$27,80",nao,,,7% OFF,Devolução grátis,https://produto.mercadolivre.com.br/MLB-291567...,Loja oficialNinja Parts SC,,...,12,32.40,pu 40 selante sc,0,0,0,0,0,0,0
2,Contra Formigas Mosquitos Baratas K-othrine Sc...,"R$12,80",nao,4.6,(876),,,https://www.mercadolivre.com.br/contra-formiga...,Vendido porDOGWORLDPETSHOP,,...,0,0.00,pu 40 selante sc,0,0,0,0,0,0,0
3,"Kelldrin Sc 25, Mata Baratas, Aranhas, Escorpiões","R$8,02",nao,4.4,(236),9% OFF,,https://www.mercadolivre.com.br/kelldrin-sc-25...,Vendido porVDAL COMERCIO,,...,0,0.00,pu 40 selante sc,0,0,0,0,0,0,0
4,Detergente Sanitário Pastilha Adesiva Citrus P...,"R$6,79",nao,4.8,(129),35% OFF,,https://www.mercadolivre.com.br/detergente-san...,Loja oficialPão de Açúcar,,...,0,0.00,pu 40 selante sc,0,0,0,0,0,0,0


In [179]:
#escluindo linhas duplicadas
df2 = df.drop_duplicates(subset=['HTML Data'])

In [180]:

df3=df2.copy()



In [181]:
df3.columns


Index(['Nome do Produto', 'Preço', 'Patrocinado', 'Avaliação',
       'Qtde avaliações:', 'Desconto', 'Frete', 'Link', 'Vendedor', 'Posição',
       'Situação', 'Quant. vendida', 'Categorias', 'Quant. fotos', 'HTML Data',
       'Valor Parcelas', 'N Parcelas', ' Total parcelas', 'Categoria',
       'Categoria0', 'Categoria1', 'Categoria2', 'Categoria3', 'Categoria4',
       'Categoria5', 'Categoria6'],
      dtype='object')

## 4.1 Criando coluna com o numero do anuncio


In [182]:
#separando link de anuncio e criando coluna anuncio 
#separando numero do anuncio 
padrao = r'/p\/([^#]+)'
padrao2 =r'(MLB-\d*)'
padrao3 = r'br\/(.*?)-'
df3['anuncio'] = df3['Link'].str.extract(padrao, expand=False)
df3['anuncio'].fillna(df3['Link'].str.extract(padrao2)[0], inplace=True)
df3['anuncio'].fillna(df3['Link'].str.extract(padrao3)[0], inplace=True)

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\383334516.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['anuncio'].fillna(df3['Link'].str.extract(padrao2)[0], inplace=True)


## 4.1 Verificando a quantidade de linhas NA na coluna anuncio 

In [183]:
df3['anuncio'].isna().sum()

0

In [184]:
#VERIFICANDO O TAMANHO DO DATAFREME
df3.shape

(46, 27)

In [185]:
#separando a coluna da data e foto 
#colunas_originais = set(df3.columns)
#def separar_conteudo(row):
#    conteudos = row['HTML Data'].strip('<p>').strip('</p>').split(', ')
#    for i, conteudo in enumerate(conteudos):
#        row[f'Conteudo_{i+1}'] = conteudo
#    return row
# #Aplicar a função em cada linha do DataFrame
#df3 = df3.apply(separar_conteudo, axis=1)
#colunas_criadas_conteudo = set(df3.columns) - colunas_originais
#colunas_criadas_conteudo

In [186]:
colunas_originais = set(df3.columns)

In [187]:
colunas_originais

{' Total parcelas',
 'Avaliação',
 'Categoria',
 'Categoria0',
 'Categoria1',
 'Categoria2',
 'Categoria3',
 'Categoria4',
 'Categoria5',
 'Categoria6',
 'Categorias',
 'Desconto',
 'Frete',
 'HTML Data',
 'Link',
 'N Parcelas',
 'Nome do Produto',
 'Patrocinado',
 'Posição',
 'Preço',
 'Qtde avaliações:',
 'Quant. fotos',
 'Quant. vendida',
 'Situação',
 'Valor Parcelas',
 'Vendedor',
 'anuncio'}

In [188]:
def separar_conteudo(row):
    html_data = row['HTML Data']
    
    # Verificar se html_data é uma lista ou string
    if isinstance(html_data, str):
        conteudos = html_data.strip('<p>').strip('</p>').split(', ')
    elif isinstance(html_data, list):
        conteudos = [item.strip('<p>').strip('</p>') for item in html_data]
    else:
        conteudos = []

    for i, conteudo in enumerate(conteudos):
        row[f'Conteudo_{i+1}'] = conteudo
    return row

# Aplicar a função em cada linha do DataFrame
df3 = df3.apply(separar_conteudo, axis=1)

# Identificar as novas colunas criadas
colunas_criadas_conteudo = set(df3.columns) - colunas_originais
colunas_criadas_conteudo

{'Conteudo_1',
 'Conteudo_10',
 'Conteudo_11',
 'Conteudo_2',
 'Conteudo_3',
 'Conteudo_4',
 'Conteudo_5',
 'Conteudo_6',
 'Conteudo_7',
 'Conteudo_8',
 'Conteudo_9'}

In [189]:
df3.columns

Index([' Total parcelas', 'Avaliação', 'Categoria', 'Categoria0', 'Categoria1',
       'Categoria2', 'Categoria3', 'Categoria4', 'Categoria5', 'Categoria6',
       'Categorias', 'Conteudo_1', 'Conteudo_10', 'Conteudo_11', 'Conteudo_2',
       'Conteudo_3', 'Conteudo_4', 'Conteudo_5', 'Conteudo_6', 'Conteudo_7',
       'Conteudo_8', 'Conteudo_9', 'Desconto', 'Frete', 'HTML Data', 'Link',
       'N Parcelas', 'Nome do Produto', 'Patrocinado', 'Posição', 'Preço',
       'Qtde avaliações:', 'Quant. fotos', 'Quant. vendida', 'Situação',
       'Valor Parcelas', 'Vendedor', 'anuncio'],
      dtype='object')

## 4.2 Criando coluna data

In [190]:
#separando e criando novas colunas com ML da foto e a data 
#padrao_5 = r'-(.*?)_'
padrao_6 =r'(?:.*?_){4}(.*?)-R'
#string_exemplo = df4['Conteudo_1']  

# Padrao regex para encontrar o que está entre '-' e '_'

for coluna in colunas_criadas_conteudo:
    # Aplicar o padrão regex na coluna 'Conteudo_1'
    #df3[coluna + '_MLB1'] = df3[coluna].str.extract(padrao_5)
    df3[coluna +'_data_MLB1'] = df3[coluna].str.extract(padrao_6)
    #transformando a coluna em formato data 
    df3[coluna +'_data_MLB1'] = pd.to_datetime(df3[coluna +'_data_MLB1'])
    df3[coluna +'_data_MLB1'] = df3[coluna +'_data_MLB1'].dt.strftime('%d/%m/%Y')

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\2466065607.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df3[coluna +'_data_MLB1'] = pd.to_datetime(df3[coluna +'_data_MLB1'])
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\2466065607.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df3[coluna +'_data_MLB1'] = pd.to_datetime(df3[coluna +'_data_MLB1'])
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\2466065607.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df3[coluna +'_data_MLB1'] = pd.to_datetime(df3[coluna +'_data_MLB1'])
C:\Users\Leticia\AppDa

In [191]:
df3.columns

Index([' Total parcelas', 'Avaliação', 'Categoria', 'Categoria0', 'Categoria1',
       'Categoria2', 'Categoria3', 'Categoria4', 'Categoria5', 'Categoria6',
       'Categorias', 'Conteudo_1', 'Conteudo_10', 'Conteudo_11', 'Conteudo_2',
       'Conteudo_3', 'Conteudo_4', 'Conteudo_5', 'Conteudo_6', 'Conteudo_7',
       'Conteudo_8', 'Conteudo_9', 'Desconto', 'Frete', 'HTML Data', 'Link',
       'N Parcelas', 'Nome do Produto', 'Patrocinado', 'Posição', 'Preço',
       'Qtde avaliações:', 'Quant. fotos', 'Quant. vendida', 'Situação',
       'Valor Parcelas', 'Vendedor', 'anuncio', 'Conteudo_1_data_MLB1',
       'Conteudo_7_data_MLB1', 'Conteudo_4_data_MLB1', 'Conteudo_9_data_MLB1',
       'Conteudo_11_data_MLB1', 'Conteudo_10_data_MLB1',
       'Conteudo_5_data_MLB1', 'Conteudo_3_data_MLB1', 'Conteudo_2_data_MLB1',
       'Conteudo_8_data_MLB1', 'Conteudo_6_data_MLB1'],
      dtype='object')

## 4.3 transformanos colunas em numero inteiros 

In [192]:
df3['Avaliação']

0        
1        
2     4.6
3     4.4
4     4.8
5        
7        
10    4.8
11       
13       
15       
17    4.6
18    4.8
19    4.8
20    4.9
21       
22    4.8
23    4.9
24    4.7
25    4.8
26    4.7
27    5.0
28    4.7
29       
30    4.7
31    4.7
32    4.7
33    4.4
34    4.7
35       
36    5.0
37    4.8
38    4.0
39       
40       
41    4.2
42    4.0
43    4.7
44    5.0
45    4.8
46    4.5
47    4.6
48    5.0
49    4.8
50    4.5
51    2.3
Name: Avaliação, dtype: object

In [193]:

# Transformando a coluna 'Avaliação' em números decimais
df3['Avaliação'] = df3['Avaliação'].replace('', 0.0)
df3['Avaliação'] = df3['Avaliação'].astype(float)

# Arredondando os valores decimais para baixo e, em seguida, convertendo para números inteiros
df3['Avaliação'] = df3['Avaliação'].apply(lambda x: int(x) if x.is_integer() else round(x))

# Convertendo outras colunas para inteiros
df3['N Parcelas'] = df3['N Parcelas'].astype(int) 
df3['Quant. fotos'] = df3['Quant. fotos'].astype(int)



In [194]:
df3['Qtde avaliações:']

0          
1          
2     (876)
3     (236)
4     (129)
5          
7          
10     (50)
11         
13         
15         
17     (36)
18     (29)
19     (23)
20     (23)
21         
22     (65)
23    (489)
24    (215)
25     (58)
26     (78)
27      (8)
28     (19)
29         
30     (39)
31      (6)
32     (12)
33     (71)
34    (147)
35         
36      (2)
37      (9)
38      (4)
39         
40         
41      (9)
42     (37)
43      (6)
44     (35)
45     (22)
46     (27)
47     (80)
48     (32)
49     (36)
50      (8)
51      (3)
Name: Qtde avaliações:, dtype: object

In [195]:
# Removendo os parênteses da coluna 'Qtde avaliações:'
df3['Qtde avaliações:'] = df3['Qtde avaliações:'].str.replace('(', '').str.replace(')', '')
#transformando em numeros inteiros
#df3['Qtde avaliações:'].fillna(0, inplace=True)
df3['Qtde avaliações:'] = df3['Qtde avaliações:'].replace('', 0)
df3['Qtde avaliações:'] = df3['Qtde avaliações:'].astype(int)

#transformando em numeros decimal 

df3['Valor Parcelas'] = df3['Valor Parcelas'].astype(float)
#df3[' Total parcelas'] = df3[' Total parcelas'].astype(float)

#Transformando a coluna Quant. vendida
df3['Quant. vendida'] = df3['Quant. vendida'].str.replace('mil', '000')
df3['Quant. vendida'] = df3['Quant. vendida'].str.replace('+', '')
df3['Quant. vendida'] = df3['Quant. vendida'].str.replace('vendido', '')
df3['Quant. vendida'] = df3['Quant. vendida'].str.replace('s', '')
df3['Quant. vendida'] = df3['Quant. vendida'].astype(int)

### 4.4 Função para retirar o R$ 

In [196]:
#Função para converter uma string de preço em reais para float
def converter_para_float(preco_string):
    # Remover caracteres não numéricos e converter para float
    preco_limpo = preco_string.replace("R$", "").replace(".", "").replace(",", ".")
    return float(preco_limpo)

#Aplicar a função para toda a coluna 'Preço'
df3['Preço'] = df3['Preço'].apply(converter_para_float)

In [197]:
df3['Preço']

0     123.40
1      27.80
2      12.80
3       8.02
4       6.79
5     952.70
7     821.29
10    101.40
11     72.00
13    157.02
15     23.69
17     19.99
18     46.77
19     53.28
20     31.81
21     26.57
22     24.00
23     45.81
24     34.85
25     21.79
26     77.00
27     24.90
28     39.80
29     39.48
30     21.89
31    113.90
32     14.63
33    132.00
34     23.50
35     17.94
36    669.00
37     17.70
38    113.00
39     50.60
40     50.97
41     46.89
42     48.37
43    205.12
44     32.46
45     38.64
46     36.90
47     36.90
48    329.00
49     17.97
50     16.52
51    246.50
Name: Preço, dtype: float64

### 4.5 Convertendo coluna total de parcela para float

In [198]:
def converter_para_float(preco):
    # Check if the value is already a float
    if isinstance(preco, float):
        return preco
    # Otherwise, clean and convert the string to float
    preco_limpo = str(preco).replace(".", "").replace(",", ".")
    return float(preco_limpo)

# Apply the function to the entire 'Total parcelas' column
df3[' Total parcelas'] = df3[' Total parcelas'].apply(converter_para_float)

### 4.6 Criando a coluna taxa


In [199]:
df3['taxas'] = 0
df3.loc[df3['Preço'] < 79.0, 'taxas'] = 6

## 4.7 Adicionando a data da raspagem

In [200]:
df3['Data_atual'] = datetime.now().date()



### 4.8 criando coluna total de preço 

In [201]:
# Verifica se ambas as colunas 'Preço' e 'Quant. vendida' contêm valores numéricos
if df3['Preço'].dtype in ['int', 'float'] and df3['Quant. vendida'].dtype == 'int':
    df3['total_de_vendas'] = df3['Preço'] * df3['Quant. vendida']
else:
    # Converta as colunas para os tipos de dados apropriados antes de realizar a multiplicação
    df3['Preço'] = pd.to_numeric(df3['Preço'], errors='coerce')
    df3['Quant. vendida'] = pd.to_numeric(df3['Quant. vendida'], errors='coerce')
    df3['total_de_vendas'] = df3['Preço'] * df3['Quant. vendida']

In [202]:
df3['total_de_vendas'].dtype

dtype('float64')

### 4.9 selecionando a menor data do anuncio 

In [203]:
df3.columns

Index([' Total parcelas', 'Avaliação', 'Categoria', 'Categoria0', 'Categoria1',
       'Categoria2', 'Categoria3', 'Categoria4', 'Categoria5', 'Categoria6',
       'Categorias', 'Conteudo_1', 'Conteudo_10', 'Conteudo_11', 'Conteudo_2',
       'Conteudo_3', 'Conteudo_4', 'Conteudo_5', 'Conteudo_6', 'Conteudo_7',
       'Conteudo_8', 'Conteudo_9', 'Desconto', 'Frete', 'HTML Data', 'Link',
       'N Parcelas', 'Nome do Produto', 'Patrocinado', 'Posição', 'Preço',
       'Qtde avaliações:', 'Quant. fotos', 'Quant. vendida', 'Situação',
       'Valor Parcelas', 'Vendedor', 'anuncio', 'Conteudo_1_data_MLB1',
       'Conteudo_7_data_MLB1', 'Conteudo_4_data_MLB1', 'Conteudo_9_data_MLB1',
       'Conteudo_11_data_MLB1', 'Conteudo_10_data_MLB1',
       'Conteudo_5_data_MLB1', 'Conteudo_3_data_MLB1', 'Conteudo_2_data_MLB1',
       'Conteudo_8_data_MLB1', 'Conteudo_6_data_MLB1', 'taxas', 'Data_atual',
       'total_de_vendas'],
      dtype='object')

In [204]:
def selecionar_colunas_por_sufixo(dataframe, sufixo):
    colunas_selecionadas = [coluna for coluna in dataframe.columns if coluna.endswith(sufixo)]
    return dataframe[colunas_selecionadas]

# Exemplo de uso:
colunas_data = selecionar_colunas_por_sufixo(df3, '_MLB1')

In [205]:
lista_colunas = colunas_data.columns.tolist()
lista_colunas

['Conteudo_1_data_MLB1',
 'Conteudo_7_data_MLB1',
 'Conteudo_4_data_MLB1',
 'Conteudo_9_data_MLB1',
 'Conteudo_11_data_MLB1',
 'Conteudo_10_data_MLB1',
 'Conteudo_5_data_MLB1',
 'Conteudo_3_data_MLB1',
 'Conteudo_2_data_MLB1',
 'Conteudo_8_data_MLB1',
 'Conteudo_6_data_MLB1']

In [206]:
df3[lista_colunas] = df3[lista_colunas].apply(pd.to_datetime)

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\1450357223.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df3[lista_colunas] = df3[lista_colunas].apply(pd.to_datetime)
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\1450357223.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df3[lista_colunas] = df3[lista_colunas].apply(pd.to_datetime)
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\1450357223.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df3[lista_colunas] = df3[lista_colunas].apply(pd.to_datetime)
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\1450357223.py:1: UserWarning: Parsing dates in %d/%m/%Y format whe

In [207]:
df3['data_anuncio'] = df3[lista_colunas].min(axis=1)

In [208]:
# Convertendo a coluna 'data_anuncio' para o tipo datetime, se não estiver
df3['data_anuncio'] = pd.to_datetime(df3['data_anuncio'])

# Substituir o dia 20 pelo dia 01
df3['data_anuncio'] = df3['data_anuncio'].apply(lambda x: x.replace(day=1) if x.day == 20 else x)

In [209]:
# Convertendo ambas as colunas para o formato datetime, se não estiverem
df3['Data_atual'] = pd.to_datetime(df3['Data_atual'])
df3['data_anuncio'] = pd.to_datetime(df3['data_anuncio'])

# Calcula a diferença em dias entre a data atual e a data do anúncio para cada linha
df3['quat_dias_do_anuncio'] = (df3['Data_atual'] - df3['data_anuncio']).dt.days
df3['quat_dias_do_anuncio'].fillna(0, inplace=True)

# Converter a coluna 'quat_dias_do_anuncio' para inteiro
df3['quat_dias_do_anuncio'].fillna(0, inplace=True)

# Converter a coluna 'quat_dias_do_anuncio' para inteiro
df3['quat_dias_do_anuncio'] = df3['quat_dias_do_anuncio'].astype(int)

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\54169396.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['quat_dias_do_anuncio'].fillna(0, inplace=True)
C:\Users\Leticia\AppData\Local\Temp\ipykernel_15464\54169396.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [210]:
df3.to_excel(caminho, index=False)